In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import svm #Import svm model
from sklearn.svm import SVC
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import LabelBinarizer
from sklearn import tree
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Read the children emergency data**


In [2]:
df=pd.read_csv('/kaggle/input/children-vitals/children_vitals_emergency_data_big_juliet.csv')
df.head()

**Lets Explore the data**

In [3]:
df.em_score.value_counts()

**Getting data mean of different emergency classes**

In [4]:
    df.groupby('em_score').mean()

In [5]:
#display basic statistics from the data
df.describe()

**Scatter plots of emergency against temp**

In [6]:
plt.scatter(x=df.temp[df.em_score==1], y=df.heart_rate[(df.em_score==1)], c="red")
plt.scatter(x=df.temp[df.em_score==0.5], y=df.heart_rate[(df.em_score==0.5)], c="blue")
plt.scatter(x=df.temp[df.em_score==0], y=df.heart_rate[(df.em_score==0)], c="green")
plt.legend(["Med_Emergency", "Moderate Emergency","Not Med_Emergency"])
plt.xlabel("Body Temperature")
plt.ylabel("Heart Rate")
plt.show()

# Using SVM to model the data

Problem Description

Predict the Children Medical Emergency based on given attributes
0 - NO EMERGECNY
0.5 - MODERATE EMERENCY
1- HIGH EMERGENCY

In [7]:
data=df.drop('em_score',axis=1)
data.head()

In [8]:
#Display basic info about the data
data.info()

In [9]:
#Separate features from th emergency output
y = data.emer_label
x = data.drop('emer_label',axis = 1) 


In [10]:
#Splitting data into training set and testing set
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7,test_size=0.3,random_state=109) # 70% training and 30% test

In [11]:
from sklearn.model_selection import GridSearchCV
#Create a svm Classifier

# defining hyperparameters range
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear']} 
  
model = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
model.fit(x_train, y_train)

In [12]:
# print best parameter after tuning
print(model.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(model.best_estimator_)

In [13]:
model=SVC(kernel='linear',C=0.1, gamma=1)
model.fit(x_train, y_train)

# SVM MODEL EVALUATION

In [14]:
#Check the performance of SVM, if less, try another classifier
#Predict the response for test dataset
y_pred = model.predict(x_test)

# Model Accuracy: how often is the classifier correct?
print("Accuracy of Support Vector Machine:",metrics.accuracy_score(y_test, y_pred))

# Decision Tree Classifier

In [15]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(x_train, y_train)

acc = dtc.score(x_test, y_test)*100

print("Decision Tree Test Accuracy {:.2f}%".format(acc))

In [16]:
#ROC Curve for the decision tree
y_pred=dtc.predict(x_test)

y_proba=dtc.predict_proba(x_test)

cm=confusion_matrix(y_test,y_pred)
print(cm)
sns.heatmap(cm,annot=True)
plt.show()

In [17]:
y_classes=['NON','MODERATE','EMERGENCY']
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(dtc, 
                   feature_names=x.columns,  
                   class_names=y_classes,
                   filled=True)

In [18]:
#Trying to predict some emergency
x=np.array([[1.0,37.5,98,110,88,46]])
x.reshape(1,-1)
y_em=dtc.predict(x)
print(y_em)

In [19]:
#Using some data to predict emergency
from sklearn.metrics import classification_report
report = classification_report(y_true=y_test, y_pred=y_pred)
print(report)